In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


clone repository and move `persian_re` package to use it

In [2]:
!git clone https://github.com/nimaafshar/persian_relation_extraction.git

Cloning into 'persian_relation_extraction'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 167 (delta 68), reused 128 (delta 39), pack-reused 10
Receiving objects: 100% (167/167), 2.28 MiB | 12.36 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [3]:
!mv persian_relation_extraction/persian_re/ persian_re/

installing specified requirements for google colab

In [4]:
!pip install -r persian_relation_extraction/requirements_colab.txt

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 316 kB 43.8 MB/s 
     |████████████████████████████████| 636 kB 45.8 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 6.5 MB 31.9 MB/s 
     |████████████████████████████████| 1.4 MB 40.4 MB/s 
     |████████████████████████████████| 233 kB 42.9 MB/s 
     |████████████████████████████████| 175 kB 43.8 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 235 kB 48.9 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=bb394c291dfcec70152028e28d1c37ccd59111701540412f6b25a8dbd1342221
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154098 sha256=87e4f5d817e

checking *CUDA* availablity

In [5]:
!nvidia-smi

Mon Mar 21 11:22:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Entity-Marker tokenizer + [CLS] output model
## Imports

In [25]:
import numpy as np

import torch, gc
import torch.nn as nn
from torch.optim import AdamW

from transformers import BertConfig
from transformers import get_linear_schedule_with_warmup

import collections

from sklearn.metrics import classification_report

In [7]:
from persian_re.preprocess import PerlexData, create_data_loader
from persian_re.tokenizers import BertEntityMarkerTokenizer
from persian_re.settings import Config
from persian_re.models import CLSModel
from persian_re.operation import Trainer,TrainingArguments

## GPU configuration

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [9]:
#clear cuda cache
gc.collect()
torch.cuda.empty_cache()

## Load Data

In [10]:
data = PerlexData.get_instance()

## Entity Marker tokenizer

In [11]:
tokenizer: BertEntityMarkerTokenizer = BertEntityMarkerTokenizer.from_pretrained(Config.MODEL_NAME_OR_PATH)

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BertEntityMarkerTokenizer'.


## Configuration

In [12]:
arguments = TrainingArguments(epochs=20,
                              clip=0.0,
                              train_callback_interval=100)

In [13]:
config = BertConfig.from_pretrained(
    Config.MODEL_NAME_OR_PATH, **{
        'label2id': data.label2ids,
        'id2label': data.id2labels,
        'hidden_dropout_prob': 0.2,
    })

In [14]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "id2label": {
    "0": "Cause-Effect(e1,e2)",
    "1": "Cause-Effect(e2,e1)",
    "2": "Component-Whole(e1,e2)",
    "3": "Component-Whole(e2,e1)",
    "4": "Content-Container(e1,e2)",
    "5": "Content-Container(e2,e1)",
    "6": "Entity-Destination(e1,e2)",
    "7": "Entity-Origin(e1,e2)",
    "8": "Entity-Origin(e2,e1)",
    "9": "Instrument-Agency(e1,e2)",
    "10": "Instrument-Agency(e2,e1)",
    "11": "Member-Collection(e1,e2)",
    "12": "Member-Collection(e2,e1)",
    "13": "Message-Topic(e1,e2)",
    "14": "Message-Topic(e2,e1)",
    "15": "Other",
    "16": "Product-Producer(e1,e2)",
    "17": "Product-Producer(e2,e1)"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Cause-Effect(e1,e2)": 0,
    "Cause-

## *DataLoader* objects

In [15]:
train_data_loader = create_data_loader(data.x_train, data.y_train, tokenizer, Config.MAX_LEN,
                                       Config.TRAIN_BATCH_SIZE, data.label2ids)
valid_data_loader = create_data_loader(data.x_valid, data.y_valid, tokenizer, Config.MAX_LEN,
                                       Config.VALID_BATCH_SIZE, data.label2ids)
test_data_loader = create_data_loader(data.x_test,data.y_test, tokenizer, Config.MAX_LEN, Config.TEST_BATCH_SIZE, data.label2ids)

## [CLS] Relation Extraction Model

In [16]:
pt_model = CLSModel(config=config)
pt_model.resize_token_embeddings(len(tokenizer))
pt_model = pt_model.to(device)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.poo

## Training
with:
- **AdamW** optimizer with initial learning rate `INITIAL_LEARNING_RATE`
- **Linear Scheduler** with no warmup
- **CrossEntropyLoss** with class weights to balance data

In [17]:
optimizer = AdamW(pt_model.parameters(), lr=Config.INITIAL_LEARNING_RATE)
total_steps = len(train_data_loader) * arguments.epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(data.class_weights).to(device))

In [18]:
trainer = Trainer(pt_model,device,loss_fn,optimizer,scheduler,arguments,train_data_loader,valid_data_loader,test_data_loader)

create dir `output` in `BASE_PATH` to save model

In [19]:
!mkdir persian_relation_extraction/output/

In [20]:
trainer.train()

Epochs... :   0%|          | 0/20 [00:00<?, ?it/s]

epoch 1/20


Training... :   0%|          | 0/1000 [00:00<?, ?it/s]

train:  [accuracy:0.5014,f1_weighted:0.4689,loss:1.5828,]


Validation... :   0%|          | 0/90 [00:00<?, ?it/s]

...

train:  [accuracy:0.9480,f1_weighted:0.9465,loss:0.1044,]


Validation... :   0%|          | 0/90 [00:00<?, ?it/s]

validation:  [accuracy:0.7183,f1_weighted:0.7192,loss:1.1353,]
epoch 6/20


Training... :   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Model Evaluation and Comparison

1. using **last epoch** (overfitted) model

In [21]:
y_pred, y_pred_probs, y_true, metrics = trainer.predict_tests()

Predicting tests:   0%|          | 0/250 [00:00<?, ?it/s]

In [24]:
metrics

{'accuracy': 0.7318659329664833, 'f1_weighted': 0.7276828777306001}

In [28]:
print(classification_report(y_true, y_pred,target_names=data.labels))

                           precision    recall  f1-score   support

      Cause-Effect(e1,e2)       0.90      0.88      0.89        99
      Cause-Effect(e2,e1)       0.85      0.85      0.85       143
   Component-Whole(e1,e2)       0.74      0.72      0.73       119
   Component-Whole(e2,e1)       0.76      0.56      0.65       110
 Content-Container(e1,e2)       0.79      0.88      0.83       113
 Content-Container(e2,e1)       0.79      0.90      0.84        29
Entity-Destination(e1,e2)       0.90      0.81      0.85       214
     Entity-Origin(e1,e2)       0.73      0.77      0.75       155
     Entity-Origin(e2,e1)       0.85      0.80      0.82        35
 Instrument-Agency(e1,e2)       0.64      0.56      0.60        16
 Instrument-Agency(e2,e1)       0.67      0.79      0.72        99
 Member-Collection(e1,e2)       0.56      0.83      0.67        24
 Member-Collection(e2,e1)       0.78      0.72      0.75       148
     Message-Topic(e1,e2)       0.81      0.85      0.83     

2. using **saved model** (which has the lowest validation loss)

In [39]:
saved_model = torch.load(Config.OUTPUT_PATH)

In [40]:
temp_trainer = Trainer(saved_model,device,loss_fn,optimizer,scheduler,arguments,train_data_loader,valid_data_loader,test_data_loader)

In [41]:
y_pred, y_pred_probs, y_true, metrics = temp_trainer.predict_tests()

Predicting tests:   0%|          | 0/250 [00:00<?, ?it/s]

In [33]:
metrics

{'accuracy': 0.688344172086043, 'f1_weighted': 0.6809201696129058}

In [34]:
print(classification_report(y_true, y_pred,target_names=data.labels))

                           precision    recall  f1-score   support

      Cause-Effect(e1,e2)       0.86      0.91      0.88        99
      Cause-Effect(e2,e1)       0.74      0.94      0.83       143
   Component-Whole(e1,e2)       0.54      0.76      0.63       119
   Component-Whole(e2,e1)       0.81      0.39      0.53       110
 Content-Container(e1,e2)       0.77      0.86      0.81       113
 Content-Container(e2,e1)       0.73      0.93      0.82        29
Entity-Destination(e1,e2)       0.89      0.74      0.81       214
     Entity-Origin(e1,e2)       0.72      0.79      0.75       155
     Entity-Origin(e2,e1)       0.50      0.80      0.62        35
 Instrument-Agency(e1,e2)       0.30      0.62      0.41        16
 Instrument-Agency(e2,e1)       0.68      0.74      0.71        99
 Member-Collection(e1,e2)       0.34      0.83      0.49        24
 Member-Collection(e2,e1)       0.83      0.58      0.68       148
     Message-Topic(e1,e2)       0.90      0.72      0.80     

3. another saved model

In [42]:
another_model = CLSModel(config=config)
another_model.resize_token_embeddings(len(tokenizer))
another_model = another_model.to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.poo

In [43]:
another_model.load_state_dict(torch.load('/content/drive/MyDrive/RE_task/all_layers_trained_model.bin'))

<All keys matched successfully>

In [44]:
temp_trainer = Trainer(another_model,device,loss_fn,optimizer,scheduler,arguments,train_data_loader,valid_data_loader,test_data_loader)

In [45]:
y_pred, y_pred_probs, y_true, metrics = temp_trainer.predict_tests()

Predicting tests:   0%|          | 0/250 [00:00<?, ?it/s]

In [46]:
metrics

{'accuracy': 0.7478739369684843, 'f1_weighted': 0.7473170516327725}

In [47]:
print(classification_report(y_true, y_pred,target_names=data.labels))

                           precision    recall  f1-score   support

      Cause-Effect(e1,e2)       0.92      0.89      0.90        99
      Cause-Effect(e2,e1)       0.87      0.86      0.86       143
   Component-Whole(e1,e2)       0.70      0.72      0.71       119
   Component-Whole(e2,e1)       0.73      0.70      0.72       110
 Content-Container(e1,e2)       0.81      0.88      0.84       113
 Content-Container(e2,e1)       0.74      0.90      0.81        29
Entity-Destination(e1,e2)       0.86      0.86      0.86       214
     Entity-Origin(e1,e2)       0.79      0.75      0.77       155
     Entity-Origin(e2,e1)       0.77      0.77      0.77        35
 Instrument-Agency(e1,e2)       0.57      0.50      0.53        16
 Instrument-Agency(e2,e1)       0.74      0.70      0.72        99
 Member-Collection(e1,e2)       0.68      0.79      0.73        24
 Member-Collection(e2,e1)       0.80      0.78      0.79       148
     Message-Topic(e1,e2)       0.80      0.86      0.83     

## Saving Best Model

In [48]:
torch.save(another_model,'/content/drive/MyDrive/RE_task/entitymarker_CLS_v0.1.bin')

## How to prevent overfitting
1. train only last n layers of BERT
2. increase dropout probablity
3. change scheduler (use `ReduceOnPlateu` or non-linear schedulers)